# Cube

$\sigma=0.1l_e$

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'cube1'
r = SampleUnion(proj_name)

canvas = [["noise", "BF", "BNF", "BGF"],
          ["vert", "edge", "area", "area_rf"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 1)
show_metrics = quick_gen_vis_type(canvas, 1)
# print(vis_type)

# exp is used for my thesis
r.show(canvas, fig_name='exp/{}.png'.format(proj_name), 
       mesh_vis_type=vis_type, 
       show_metrics=show_metrics,
       titles=canvas, zoom=1.4)

pyvista 的 [shadow](https://pyvista-experimental-dynamic.netlify.app/examples/04-lights/shadows) 不好，没有软阴影的。尝试了一下，改成自定义光源+开SSAO的方式

https://docs.pyvista.org/version/stable/api/plotting/_autosummary/pyvista.plotter.enable_ssao
s

pip install opencv-python -i https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
def render_test(mesh, 
                        zoom=5, 
                        mesh_color='white',
                        feature_color='k',
                        feature_line_width=5,
                        render_smooth=False,
                        render_sharp=False,
                        name='vert',
                        folder='cube',
                        resolution=800,
                        floor_up = 0.,
                        ssao_radius=2,
                        ssao_bias=0.5,
                        ssao_kernel_size=256,
                        use_shadow = False):
    fig_name = 'exp/'+folder
    if not os.path.exists(fig_name):
        os.mkdir(fig_name)
    fig_name = fig_name+'/'+name+'.png'
    

    # Add a thin box below the mesh
    bounds = mesh.bounds
    rnge = (bounds[1] - bounds[0], bounds[3] - bounds[2], bounds[5] - bounds[4])

    expand = 5.0
    height = rnge[2] * floor_up
    center = np.array(mesh.center)
    center -= [0, 0, mesh.center[2] - bounds[4] + height / 2]

    width = rnge[0] * (1 + expand)
    length = rnge[1] * (1 + expand)
    base_mesh = pyvista.Cube(center, width, length, height)

    # rotate base and mesh to get a better view
    #base_mesh.rotate_z(30, inplace=True)
    #mesh.rotate_z(30, inplace=True)

    light1 = pyvista.Light(
        position=(0, rnge[1]*10, 0),
        focal_point=(0, 0, 0),
        #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
        color=[1.0, 1.0, 1.0, 1.0],
        intensity=1.1
        ,
    )

    light2 = pyvista.Light(
        position=(0, 0, rnge[2]*10),
        focal_point=(0, 0, 0),
        #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
        color=[1.0, 1.0, 1.0, 1.0],
        intensity=1.1
        ,
    )

    light3 = pyvista.Light(
        position=(rnge[0]*10, 0, 0.0),
        focal_point=(0, 0, 0),
        #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
        color=[1.0, 1.0, 1.0, 1.0],
        intensity=1.1
        ,
    )

    # create the plotter with custom lighting
    """"
    pl = pyvista.Plotter(lighting=None, window_size=(resolution, resolution), off_screen=True)
    pl.add_light(light1)
    pl.add_light(light2)
    pl.add_light(light3)
    
    pl.add_mesh(
        mesh,
        color=mesh_color,
        smooth_shading=render_smooth, 
        split_sharp_edges=render_sharp
    )
    """
    pl = pyvista.Plotter(window_size=(resolution, resolution), off_screen=True)
    pl.add_mesh(
        mesh,
        color=mesh_color,
        smooth_shading=render_smooth, 
        split_sharp_edges=render_sharp
    )
    pl.add_mesh(base_mesh, color='lightblue')
    edges = mesh.extract_feature_edges(
    boundary_edges=False, non_manifold_edges=False, feature_angle=70, manifold_edges=False
)
    #pl.add_mesh(edges, color=feature_color, line_width=feature_line_width)
    if use_shadow:
        pl.enable_shadows()
    pl.enable_ssao(radius=ssao_radius, bias=ssao_bias, kernel_size=ssao_kernel_size, blur=True)
    pl.enable_anti_aliasing('ssaa')
    pl.camera.zoom(zoom)
    pl.show(screenshot=fig_name)
    pl.close()

proj_name = 'selected'
r = SampleUnion(proj_name)

name='block'
mesh = r.results[name].denoised_mesh 
render_test(mesh)


In [ ]:
def render_scene1(mesh):
    lightfac = 1.2
    light1 = pyvista.Light(
        position=(0, lightfac*1.0, 0),
        focal_point=(0, 0, 0),
        #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
        color=[1.0, 1.0, 1.0, 1.0],
        intensity=0.8
        ,
    )

    light2 = pyvista.Light(
        position=(0, 0, lightfac*1.0),
        focal_point=(0, 0, 0),
        #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
        color=[1.0, 1.0, 1.0, 1.0],
        intensity=1.1
        ,
    )

    light3 = pyvista.Light(
        position=(lightfac*1.0, 0, 0.0),
        focal_point=(0, 0, 0),
        #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
        color=[1.0, 1.0, 1.0, 1.0],
        intensity=0.8
        ,
    )

    # Add a thin box below the mesh
    bounds = mesh.bounds
    rnge = (bounds[1] - bounds[0], bounds[3] - bounds[2], bounds[5] - bounds[4])

    expand = 5.0
    height = rnge[2] * 0.05
    center = np.array(mesh.center)
    center -= [0, 0, mesh.center[2] - bounds[4] + height / 2]

    width = rnge[0] * (1 + expand)
    length = rnge[1] * (1 + expand)
    base_mesh = pyvista.Cube(center, width, length, height)

    # rotate base and mesh to get a better view
    #base_mesh.rotate_z(30, inplace=True)
    #mesh.rotate_z(30, inplace=True)

    # create the plotter with custom lighting
    pl = pyvista.Plotter(lighting=None, window_size=(800, 800))
    pl.add_light(light1)
    pl.add_light(light2)
    pl.add_light(light3)
    pl.add_mesh(
        mesh,
        ambient=0.2,
        diffuse=0.5,
        specular=0.4,
        specular_power=50,
        #smooth_shading=True,
        color='white',
    )
    pl.add_mesh(base_mesh, color='white')
    #pl.enable_shadows()
    pl.enable_ssao(radius=0.6, bias=0.05, kernel_size=256, blur=True)
    edges = mesh.extract_feature_edges(
    boundary_edges=False, non_manifold_edges=False, feature_angle=150, manifold_edges=False
)
    pl.add_mesh(edges, color='purple', line_width=5)
    pl.camera.zoom(5)
    pl.show()
    pl.close()

In [ ]:
import pyvista as pv
mesh = r.results['BNF'].denoised_mesh
print(mesh)
mesh = r.results['edge'].denoised_mesh
print(mesh)
#mesh = r.results['noise'].denoised_mesh


render_scene1(mesh)



pip install bpy -i https://pypi.tuna.tsinghua.edu.cn/simple


$\sigma=0.5l_e$

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'cube5'
r = SampleUnion(proj_name)

canvas = [["noise", "BF", "BNF", "BGF"],
          ["vert", "edge", "area", "area_rf"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 1)
show_metrics = quick_gen_vis_type(canvas, 1)
# print(vis_type)

r.show(canvas, fig_name='exp/{}.png'.format(proj_name), 
       mesh_vis_type=vis_type, 
       show_metrics=show_metrics,
       titles=canvas)

In [ ]:
import pyvista as pv
mesh = r.results['area_rf'].denoised_mesh
print(mesh)
#mesh = r.results['edge'].denoised_mesh
print(mesh)
#mesh = r.results['noise'].denoised_mesh


render_scene1(mesh)

$\sigma=0.9l_e$

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'cube9'
r = SampleUnion(proj_name)

canvas = [["noise", "BF", "BNF", "BGF"],
          ["vert", "edge", "area", "area_rf"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 1)
show_metrics = quick_gen_vis_type(canvas, 1)
# print(vis_type)

r.show(canvas, fig_name='exp/{}.png'.format(proj_name), 
       mesh_vis_type=vis_type, 
       show_metrics=show_metrics,
       titles=canvas)

# Selected Figures

## Synthetic

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'selected'
r = SampleUnion(proj_name)
"""
for j in r.results:
    print(j)
"""

canvas = [["part_Lp", "trim-star", "fandisk", "octa-flower", "block"] for i in range(3)] # must be nxm

vis_type = quick_gen_vis_type(canvas, [[-2], [0], [1]])
show_metrics = quick_gen_vis_type(canvas, [[0], [0], [1]])
# print(vis_type)

r.show(canvas, fig_name='exp/{}-syn1.png'.format(proj_name), 
       mesh_vis_type=vis_type, show_metrics=show_metrics)

In [11]:
import pyvista
import os
import numpy as np
import math
import copy
from PIL import Image
import cv2
from scripts.visualize import *

class RenderBase:
    def __init__(self, union):
        self.union = union 
        self.names = ['noise','denoised', 'gt' ]

        self.use_ao = False
        self.use_ground = False

    def set_mesh(self, name, 
                 mesh_color = 'white', 
                 render_smooth=False,
                 render_sharp=False,
                 resolution=800):
        # set path
        self.name = name 
        render_dir = 'exp/' + self.name
        self.render_dir = render_dir
        if not os.path.exists(render_dir):
            os.mkdir(render_dir)
        self.fig_names = []
        self.fig_names_ = []
        for i in self.names:
            self.fig_names.append(render_dir + '/' + i + '.png')
            self.fig_names_.append(render_dir + '/' + i + '_.png')
        self.cat_file = render_dir + '/out.png'

        # set mesh
        union = self.union
        assert name in union.results
        mesh0 = union.results[name].noise_mesh 
        mesh1 = union.results[name].denoised_mesh 
        mesh2 = union.results[name].gt_mesh 
        self.mesh = [mesh0, mesh1, mesh2]
        self.resolution = resolution
        

        # set plotter
        pl_n = pyvista.Plotter(window_size=(resolution, resolution), off_screen=True)
        pl_d = pyvista.Plotter(window_size=(resolution, resolution), off_screen=True)
        pl_g = pyvista.Plotter(window_size=(resolution, resolution), off_screen=True)
        self.pl = [pl_n, pl_d, pl_g]

        for i in range(3):
            mesh = self.mesh[i]
            pl = self.pl[i]

            pl.add_mesh(
                mesh,
                ambient=0.2,
                diffuse=0.5,
                specular=0.4,
                specular_power=50,
                color=mesh_color,
                smooth_shading=render_smooth, 
                split_sharp_edges=render_sharp
            )

            if self.use_ground:
                bounds = mesh.bounds
                rnge = (bounds[1] - bounds[0], bounds[3] - bounds[2], bounds[5] - bounds[4])

    def append_ground(self,
                    expand = 7,
                    floor_up = 0.1,
                    ground_color = None):
        # use for advance rendering
        self.use_ground = True
        self.ground_expand = expand
        for i in range(3):
            pl = self.pl[i]
            mesh = self.mesh[i]

            bounds = mesh.bounds
            rnge = (bounds[1] - bounds[0], bounds[3] - bounds[2], bounds[5] - bounds[4])

            height = rnge[2] * floor_up
            center = np.array(mesh.center)
            center -= [0, 0, mesh.center[2] - bounds[4] + height / 2]

            width = rnge[0] * (1 + expand)
            length = rnge[1] * (1 + expand)
            base_mesh = pyvista.Cube(center, width, length, height)
            pl.add_mesh(base_mesh, color=ground_color)

    def append_light(self, strength1=0.8, strength2=1.1,  strength3=0.8):
        for i in range(3):
            pl = self.pl[i]
            mesh = self.mesh[i]

            bounds = mesh.bounds
            rnge = (bounds[1] - bounds[0], bounds[3] - bounds[2], bounds[5] - bounds[4])

            light1 = pyvista.Light(
                position=(0, rnge[1], 0),
                focal_point=(0, 0, 0),
                #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
                color=[1.0, 1.0, 1.0, 1.0],
                intensity=strength1
                ,
            )

            light2 = pyvista.Light(
                position=(0, 0, rnge[2]),
                focal_point=(0, 0, 0),
                #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
                color=[1.0, 1.0, 1.0, 1.0],
                intensity=strength2
                ,
            )

            light3 = pyvista.Light(
                position=(rnge[0], 0, 0.0),
                focal_point=(0, 0, 0),
                #color=[1.0, 0.83921, 0.6666, 1.0],  # Color temp. 2850 K
                color=[1.0, 1.0, 1.0, 1.0],
                intensity=strength3
                ,
            )

            pl.add_light(light1)
            pl.add_light(light2)
            pl.add_light(light3)



    def append_feature_line(self,
                            feature_angle=70,
                            feature_color='red',
                            feature_line_width=5):
        for i in range(3):
            pl = self.pl[i]
            mesh = self.mesh[i]
            

            edges = mesh.extract_feature_edges(
                    boundary_edges=False, non_manifold_edges=False, feature_angle=feature_angle, manifold_edges=False
                )
            pl.add_mesh(edges, color=feature_color, line_width=feature_line_width)

    def enable_ao(self, ssao_radius=0.9, ssao_bias=0.05, ssao_kernel_size=256):
        for i in range(3):
            pl = self.pl[i]
            pl.enable_ssao(radius=ssao_radius, bias=ssao_bias, kernel_size=ssao_kernel_size, blur=True)
            pl.enable_anti_aliasing('ssaa')

    def render_norm(self, show_bar=True, sargs=None, cmap="jet"):
        mesh = self.union.results[self.name].visual_norm()
        resolution = self.resolution
        zoom = self.zoom
        camera_position = self.camera_position
        fig_name = self.render_dir + '/norm.png'
        pl = pyvista.Plotter(window_size=(resolution, resolution), off_screen=True)

        if sargs==None:
            # https://docs.pyvista.org/version/stable/examples/02-plot/scalar-bars
            sargs = dict(height=0.2, vertical=True, position_x=0.05, position_y=0.05, n_labels=2)

        pl.add_mesh(
                    mesh,
                    scalar_bar_args=sargs,
                    cmap=cmap,
                    show_scalar_bar=show_bar
                )

        pl.camera.zoom(zoom)
        if camera_position!=None:
            pl.camera_position = camera_position
        pl.screenshot(fig_name)
        pl.close()
        self.fig_names_.append(fig_name)

    def concat(self, paths=None, save_file=None, vertical=False):
        if paths==None:
            paths=self.fig_names_ 
            save_file = self.cat_file
        img_array = ''
        img = ''
        for i, v in enumerate(paths):
            if i == 0:
                img = Image.open(v)  # 打开图片
                img_array = np.array(img)  # 转化为np array对象
            if i > 0:
                img_array2 = np.array(Image.open(v))
                if vertical:
                    img_array = np.concatenate((img_array, img_array2), axis=0)
                else:
                    img_array = np.concatenate((img_array, img_array2), axis=1)  # 横向拼接
                img = Image.fromarray(img_array)
        img.save(save_file)

        
    def render(self,
            zoom=None,
            camera_position=None,
            cx = 0.5,
            cy = 0.8,
            dx = 0.1,
            dy = 0.1,
            upscale = 1.7,
            ):
        
        if zoom==None:
            zoom = 1
            if self.use_ground:
                zoom = self.ground_expand-2
        
        # record for render_norm()
        self.zoom = zoom
        self.camera_position=camera_position

        for i in range(3):
            fig_name = self.fig_names[i]
            fig_name_ = self.fig_names_[i]
            pl = self.pl[i]
            

            pl.camera.zoom(zoom)
            if camera_position!=None:
                pl.camera_position = camera_position
            pl.screenshot(fig_name)
            pl.close()

            # post process
            img = cv2.imread(fig_name)
            resulution_x = img.shape[1]
            resulution_y = img.shape[0]

            x = int(cx*resulution_x)
            y = int(cy*resulution_y)

            upscale = 1.7

            bound = int(resulution_x/80)

            part_left = int(x - dx*resulution_x)
            part_right = int(x + dx*resulution_x)
            part_top = int(y - dy*resulution_y)
            part_bottom = int(y + dy*resulution_y)

            loc_left = resulution_x - int(dx*resulution_x*2*upscale) - bound
            loc_top = bound
            big_length = int(dx*resulution_x*2*upscale)
            loc_right = loc_left + big_length
            loc_bottom = loc_top + big_length

            part = img[part_top:part_bottom,part_left:part_right]
            mask = cv2.resize(part, (big_length, big_length), fx=0, fy=0, interpolation=cv2.INTER_LINEAR)

            img[loc_top:loc_bottom,loc_left:loc_right]=mask

            cv2.rectangle(img,(part_left,part_top),(part_right,part_bottom),(0,0,0),2)
            cv2.rectangle(img,(loc_left,loc_top),(loc_right,loc_bottom),(0,0,0),2)
            #cv2.line(img,(part_right,part_top),(loc_right,loc_top),(0,0,0),2)
            #cv2.line(img,(part_left,part_bottom),(loc_left,loc_bottom),(0,0,0),2)

            cv2.imwrite(fig_name_, img)
        








In [12]:
proj_name = 'selected'
r = SampleUnion(proj_name)

for i in r.results:
    print(i)

rr = RenderBase(r)


part_Lp
trim-star
fandisk
octa-flower
block
fertility
joint
bunny
chinese_lion
dragon
big_girl_01
cone_01
david_01
pyramid_01
boy_01
big_girl_03
cone_03
david_03
pyramid_03
boy_03
big_girl
cone
girl
boy02
david


In [15]:
sargs = dict(height=0.2, vertical=True, position_x=0.9, position_y=0.75, n_labels=2)

rr.set_mesh('part_Lp')
rr.append_feature_line()
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render()
rr.render_norm(sargs = sargs)
rr.concat()

In [19]:
rr.set_mesh('trim-star')
rr.append_feature_line()
rr.append_light(strength1=0.3, strength2=0.4, strength3=0.3)
rr.enable_ao()
rr.render(cy=0.5,cx=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [20]:
rr.set_mesh('fandisk')
rr.append_feature_line()
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(cy=0.4,cx=0.18)
rr.render_norm(sargs = sargs)
rr.concat()

In [21]:
rr.set_mesh('octa-flower')
rr.append_feature_line()
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(cy=0.5,zoom=1.2)
rr.render_norm(sargs = sargs)
rr.concat()

In [22]:
rr.set_mesh('block')
rr.append_feature_line()
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(cx=0.3, cy=0.6)
rr.render_norm(sargs = sargs)
rr.concat()

In [23]:
rr.set_mesh('fertility')
#rr.append_feature_line() # no feature line here
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [46]:
rr.set_mesh('joint')
rr.append_feature_line() 
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(cy=0.35, cx=0.4)
rr.render_norm(sargs = sargs)
rr.concat()

In [48]:
rr.set_mesh('bunny')
#rr.append_feature_line() # no feature line here
rr.append_light(strength1=6, strength2=6, strength3=6)
#rr.append_ground()
rr.enable_ao()
rr.render(cy=0.65, cx=0.75)
rr.render_norm(sargs = sargs)
rr.concat()

In [54]:
rr.set_mesh('chinese_lion')
rr.append_feature_line()
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(cx=0.5, cy=0.5, zoom=1.4)
rr.render_norm(sargs = sargs)
rr.concat()

In [55]:
rr.set_mesh('dragon')
rr.append_feature_line()
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(cx=0.5, cy=0.5, zoom=1.4)
rr.render_norm(sargs = sargs)
rr.concat()

## Kinect v1

In [24]:
kv1_color = 'lightblue'
rr.set_mesh('big_girl_01', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [25]:
rr.set_mesh('cone_01', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [26]:
rr.set_mesh('david_01', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [27]:
rr.set_mesh('pyramid_01', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

## Kinect v2

In [28]:
kv2_color = 'lightblue'
rr.set_mesh('big_girl_03', mesh_color=kv2_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [29]:
rr.set_mesh('cone_03', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [30]:
rr.set_mesh('david_03', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [31]:
rr.set_mesh('pyramid_03', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

## Kinect Fusion

In [ ]:
canvas = [["big_girl", "cone", "girl", "boy02", "david"] for i in range(3)] # must be nxm

vis_type = quick_gen_vis_type(canvas, [[-2], [0], [1]])
show_metrics = quick_gen_vis_type(canvas, [[0], [0], [1]])
# print(vis_type)

r.show(canvas, fig_name='exp/{}-kf.png'.format(proj_name), 
       mesh_vis_type=vis_type, show_metrics=show_metrics, camera_position='xy')

In [32]:
rr.set_mesh('big_girl', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [33]:
rr.set_mesh('cone', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [34]:
rr.set_mesh('girl', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [35]:
rr.set_mesh('boy02', mesh_color=kv1_color)
rr.append_feature_line()  
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

In [36]:
rr.set_mesh('david', mesh_color=kv1_color)
rr.append_feature_line() 
rr.append_light(strength1=0.4, strength2=0.6, strength3=0.4)
#rr.append_ground()
rr.enable_ao()
rr.render(camera_position='xy', cy=0.5)
rr.render_norm(sargs = sargs)
rr.concat()

# 非规整网格

In [ ]:
import os
import copy
from scripts.visualize import *

proj_name = 'ir1'
r = SampleUnion(proj_name)


canvas = [["40359", "40359edge", "40359area", "40359area_r"],
             ["40447", "40447edge", "40447area", "40447area_r"],
             ["40843", "40843edge", "40843area", "40843area_r"],
             ["43960", "43960edge", "43960area", "43960area_r"]] # must be nxm

vis_type = quick_gen_vis_type(canvas, 0)
show_metrics = quick_gen_vis_type(canvas, 1)

r.show(canvas, fig_name='exp/{}.png'.format(proj_name), mesh_vis_type=vis_type, show_metrics=show_metrics)


# Robust

通过以下脚本简单制作`test_robust.bat`中的加权参数（如果需要修改的话运行）

为了美观，考虑使用以下配色 https://zhuanlan.zhihu.com/p/601170660 


In [ ]:
"""
for i in range(1,41):
    print('set m{}={:.1f}'.format(i,i*0.1))
"""

color1 = (20/255,54/255,95/255) 
color2 = (214/255,79/255,56/255)

color3 = (118/255,162/255,185/255)
color4 = (191/255,217/255,229/255)
color5 = (248/255,242/255,236/255)

## Block ($\sigma=0.2l_e$)

对CAD模型的测试（一般来说CAD模型的超参数和非CAD模型的超参数不一致）

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scripts.visualize import *

def gen_robust_figure(proj_name, xlim=(0.6, 1.5), ylim=(2.55, 2.75),
                      loc=(0.2, 0.4, 0.4, 0.4),
                      bbox_color = (191/255,217/255,229/255)):
    r = SampleUnion(proj_name)
    AADs = []
    AHDs = []
    for k in r.results:
        AADs.append(r.results[k].AAD)
        AHDs.append(r.results[k].AHD*1000)
    

    # visual settings
    font = {'family' : 'Times New Roman',
            'size'   : 12}
    plt.rc('font', **font)

    # ---------------------------------------------------------------------------------
    # https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html #
    # ---------------------------------------------------------------------------------

    # Create some mock data
    t = np.array([0.1*i for i in range(1, len(AADs)+1)])
    data1 = np.array(AADs)
    data1_ = 'Average Angular Distance'
    data2 = np.array(AHDs)
    data2_ = 'Average Hausdorff Distance (10-3)'

    fig, ax1 = plt.subplots()

    color1 = (20/255,54/255,95/255)
    ax1.set_xlabel("multiplicator", fontsize=12)
    ax1.set_ylabel(data1_, color=color1)
    ax1.plot(t, data1, color=color1)
    ax1.tick_params(axis='y', labelcolor='black')

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color2 = (214/255,79/255,56/255)
    ax2.set_ylabel(data2_, color=color2)  # we already handled the x-label with ax1
    ax2.plot(t, data2, color=color2)
    ax2.tick_params(axis='y', labelcolor='black')

    fig.tight_layout()  # otherwise the right y-label is slightly clipped

    #-----------------------------------------
    # https://zhuanlan.zhihu.com/p/136574534 #
    #-----------------------------------------

    axins = ax1.inset_axes(loc)
    axins.plot(t, data1, color=color1)


    # 调整子坐标系的显示范围
    axins.set_xlim(xlim[0], xlim[1])
    axins.set_ylim(ylim[0], ylim[1])

    mark_inset(ax1, axins, loc1=3, loc2=4, fc="none", ec=bbox_color, lw=1)
    plt.savefig('exp/{}.png'.format(proj_name), dpi=300, bbox_inches='tight')
    plt.show()

gen_robust_figure('robust_block')


## nicolo

In [ ]:
gen_robust_figure('robust_nicolo', xlim=(0.3,0.9), ylim=(6.4,7), 
                  loc=(0.5, 0.2, 0.4, 0.3))

## 不同编译选项对速度的影响

绮良良的颜色

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c1,c2,c3,c4,c5])         

data = {'O0':[22568.2,    23011.9,  23018.7,  23122.7, 22870.3], 
        'O1':[3045.54,    3071.12,  3031.58,  3098.17, 3022.1],
        'O2':[3011.35,    3040.07,  2953.53,  3125.52, 2953.05],
        'O3':[2950.83,    3026.71,  2925.76,  3163.12, 2940.22],
        'Ofast':[2957.44, 2996.12,  3023.72,  3063.46, 2949.75]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.barplot(df,estimator="mean",errorbar=None, palette=cc)

for p in ax.patches:
    ax.annotate(' {:.1f}'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+100), color='black', size=10)
plt.ylabel("Time (ms)", fontsize=12)
plt.savefig('exp/compiler_opt1.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c1,c2,c3,c4,c5])         

data = {'O0':[8606], 
        'O1':[964],
        'O2':[892],
        'O3':[915],
        'Ofast':[916]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.barplot(df,estimator="mean",errorbar=None, palette=cc)

for p in ax.patches:
    ax.annotate(' {:.0f}'.format(int(p.get_height())), (p.get_x()+0.1, p.get_height()+100), color='black', size=10)
plt.ylabel("Memory (KB)", fontsize=12)
plt.savefig('./exp/compiler_opt_m.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c2,c3,c4,c5])   

data = {#'O0':[22568.2,    23011.9,  23018.7,  23122.7, 22870.3], 
        'O1':[3045.54,    3071.12,  3031.58,  3098.17, 3022.1],
        'O2':[3011.35,    3040.07,  2953.53,  3125.52, 2953.05],
        'O3':[2950.83,    3026.71,  2925.76,  3163.12, 2940.22],
        'Ofast':[2957.44, 2996.12,  3023.72,  3063.46, 2949.75]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

plt.ylim(2800, 3150)
plt.ylabel("Time (ms)", fontsize=12)
sns.barplot(df, errorbar="sd", capsize=0.2, palette=cc)
plt.savefig('exp/compiler_opt2.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c2,c3,c4])   

data = {'mavx mfma':[3005.19,    2980.16,  2984.61, 3000.06, 2980.52], 
        'openmp':[2947,    2914.32,  2939.02,  2917.64, 2934.83],
        'mavx mfma + openmp':[2943.69,    2940.34,  2950.99, 2993.23, 2960]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

plt.ylim(2900, 3020)
plt.ylabel("Time (ms)", fontsize=12)
sns.barplot(df, errorbar="sd", capsize=0.2, palette=cc)
plt.savefig('exp/compiler_opt_openmp.png', dpi=300, bbox_inches='tight')
plt.show()

# 算法复杂度-可视化

In [ ]:
from scripts.visualize import *

proj_name = 'selected'
r = SampleUnion(proj_name)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

x_values = []
y_values = []

for j in sorted(r.results.keys(), key=lambda x: r.results[x].gt_mesh.n_points):
    sample = r.results[j]
    v = sample.gt_mesh.n_points * 1e-3 # -> K 
    t = sample.time * 1e-3 # ms -> s
    x_values.append(v)
    y_values.append(t)

x_data = np.array(x_values)
y_data = np.array(y_values)

def power_function(x, a, b):
    return a * np.power(x, b)

params, covariance = curve_fit(power_function, x_data, y_data)

a_fit, b_fit = params

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)
plt.grid(True)

plt.scatter(x_data, y_data, label='Original data')
x_fit = np.linspace(min(x_data), max(x_data), 100)
plt.plot(x_fit, power_function(x_fit, a_fit, b_fit), color='red', label='Fitted curve')

plt.savefig('exp/compiler_opt2.png', dpi=300, bbox_inches='tight')
plt.xlabel('Vertice nums (K)')
plt.ylabel('Time (s)')
plt.title('Algorithm Time Complexity')

plt.savefig('exp/time_complexity_o2.png', dpi=300, bbox_inches='tight')
plt.show()


print("power_fun a:", a_fit)
print("power_fun b:", b_fit)

# 数值求解器

## 随着HQS迭代，每个step中求解器所需的迭代次数

In [ ]:
import os
import copy
from scripts.visualize import *

def parse(line):
    # NAME:'xxx \n' -> 'xxx'
    # return line.split()[0].split(':')[1]
    return line.split(':')[1].split()[0]



def get_iter_steps(proj_name):
    work_dir = './run/{}/'.format(proj_name)
    assert os.path.exists(work_dir), "Please ensure this job exists in ./run"
    time_file = work_dir + 'time.txt'

    f=open(time_file)
    data = f.readlines() 
    f.close()

    dim0 = []
    dim1 = []
    dim2 = []

    outer = 0
    name = None
    inner = 0

    while(outer<len(data)):
        line = data[outer]
        if line.startswith('Iteration'):
            num = parse(line)
            if inner%3 == 0:
                dim0.append(int(num))
            elif inner%3 == 1:
                dim1.append(int(num))
            else:
                dim2.append(int(num))  
            inner += 1         
        outer += 1
    return dim0, dim1, dim2 


# r = SampleUnion(proj_name)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


i0, _, _ = get_iter_steps('iter-block-0')
i1, _, _ = get_iter_steps('iter-block-1')
i2, _, _ = get_iter_steps('iter-block-2')
i3, _, _ = get_iter_steps('iter-block-3')
i4, _, _ = get_iter_steps('iter-block-4')
i5, _, _ = get_iter_steps('iter-block-5')

c0 = (239,145,99)
c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c0,c1,c2,c3,c4,c5])         

data = {'Jacobi':i0, 
        'Gauss-Seidel':i1,
        'CG':i2,
        'BiCGSTAB':i3,
        'MINRES':i4,
        'GMRES':i5
        }
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.lineplot(df, palette=cc)
ax.set_ylim(0,1200)

plt.ylabel("Solver's iters", fontsize=12)
plt.savefig('exp/iter_iters.png', dpi=300, bbox_inches='tight')
plt.show()

## 每种Solver固定不同的迭代次数，其对应结果的AAD指标

In [ ]:

r0 = SampleUnion('iter-block-0')
r1 = SampleUnion('iter-block-1')
r2 = SampleUnion('iter-block-2')
r3 = SampleUnion('iter-block-3')
r4 = SampleUnion('iter-block-4')
r5 = SampleUnion('iter-block-5')

Rs = [r0, r1, r2, r3, r4, r5]
As = [[],[],[],[],[],[]]

for i in range(1,41):
    for j in range(6):
        As[j].append(Rs[j].results['block{}'.format(i)].AAD)



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c0 = (239,145,99)
c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c0,c1,c2,c3,c4,c5])         

data = {'Jacobi':As[0], 
        'Gauss-Seidel':As[1],
        'CG':As[2],
        'BiCGSTAB':As[3],
        'MINRES':As[4],
        'GMRES':As[5]
        }
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.lineplot(df, palette=cc)
ax.set_ylim(2.5,4.5)

plt.ylabel("Average Angular Distance", fontsize=12)
plt.savefig('exp/iter_aads.png', dpi=300, bbox_inches='tight')
plt.show()

## 每种Solver固定不同的迭代次数，其所需要的时间

In [ ]:
r0 = SampleUnion('iter-block-0')
r1 = SampleUnion('iter-block-1')
r2 = SampleUnion('iter-block-2')
r3 = SampleUnion('iter-block-3')
r4 = SampleUnion('iter-block-4')
r5 = SampleUnion('iter-block-5')

Rs = [r0, r1, r2, r3, r4, r5]
As = [[],[],[],[],[],[]]

for i in range(1,41):
    for j in range(6):
        As[j].append(Rs[j].results['block{}'.format(i)].time)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c0 = (239,145,99)
c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c0,c1,c2,c3,c4,c5])         

data = {'Jacobi':As[0], 
        'Gauss-Seidel':As[1],
        'CG':As[2],
        'BiCGSTAB':As[3],
        'MINRES':As[4],
        'GMRES':As[5]
        }
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.lineplot(df, palette=cc)
#ax.set_ylim(2.5,4.5)

plt.ylabel("Time (ms)", fontsize=12)
plt.savefig('exp/iter_time.png', dpi=300, bbox_inches='tight')
plt.show()

## 流程图

为得到矢量图，打开https://mermaid.live/edit，将代码复制上去，即可从网页端下载

In [ ]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

mm("""
sequenceDiagram
    participant CPU
    participant GPU
    #autonumber
    note over CPU: 读取网格V,F
    note over CPU: 计算边邻接矩阵E
    CPU-->> GPU: 分配V,E的显存空间;
    CPU ->> GPU: 拷贝V,E到显存;  
    note over GPU: 分配P,δ的显存空间;
    note over GPU: 将P初始化为V;
    note over CPU: 初始化HQS设置;
    loop HQS β<βmax
        note over GPU: 【自定义核函数】<br/>基于P,E并行构造拉普拉斯算子L<br/>以CSR格式在显存中表示;
        note over CPU:  β← 2β;
        Note over CPU,GPU: 局部求解
        note over GPU: 【cuSPARSE】<br/>δ=LP;
        note over GPU: 【自定义核函数】<br/>并行对δ进行掩码操作;
        Note over CPU,GPU: 全局求解
        note over GPU: 【cuSPARSE】<br/>计算系数矩阵 A=βLTL + I;
        note over GPU: 【cuSPARSE】<br/>计算常数向量 b=βLTδ + V;

        loop CG迭代求解
          CPU->GPU: 【cuSPARSE, cuBLAS】<br/> 限定CG迭代次数为4 <br/>更新P
        end
    end
    GPU ->> CPU: 拷贝P到内存; 
    note over GPU: 释放所有显存空间;
    note over CPU: 保存网格P,F到文件;
""")

# GPU 展示

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use("ggplot")

platform = ["CPU", "GPU"]
cpu = np.array([1904.24, 1904.24])
hqs = np.array([23141.6, 1798.98])
gpuio = np.array([0, 2560.23-1798.98])

cpu/=1000
hqs/=1000
gpuio/=1000

fig, ax = plt.subplots(figsize=(10, 7))

ax.bar(platform, cpu, color="red", label="CPU preprocess")

ax.bar(platform, hqs, color="blue", bottom=cpu, label="HQS")

ax.bar(platform, gpuio, color="green", 
       bottom=np.array(hqs) + np.array(cpu), label="GPU IO")

ax.set_title("Merlion", fontsize=15)
#ax.set_xlabel("")
ax.set_ylabel("Time (s)")
ax.set_ylim(0,26)
ax.legend()
plt.savefig('exp/gpu-merlin.png', dpi=300, bbox_inches='tight')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use("ggplot")

platform = ["CPU", "GPU"]
cpu = np.array([64.2884, 64.2884])
hqs = np.array([774.389, 187.495])
gpuio = np.array([0, 899.668-187.495])



fig, ax = plt.subplots(figsize=(10, 7))

ax.bar(platform, cpu, color="red", label="CPU preprocess")

ax.bar(platform, hqs, color="blue", bottom=cpu, label="HQS")

ax.bar(platform, gpuio, color="green", 
       bottom=np.array(hqs) + np.array(cpu), label="GPU IO")

ax.set_title("Block", fontsize=15)
#ax.set_xlabel("")
ax.set_ylabel("Time (ms)")
ax.set_ylim(0,1000)
ax.legend()
plt.savefig('exp/gpu-block.png', dpi=300, bbox_inches='tight')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c1 = (53,93,115)
c2 = (141,192,200)
c3 = (213,199,172)
c4 = (234,199,114)
c5 = (105,169, 78)

def gen_c(colors):
    ret = []
    for color in colors:
        c = (color[0]/255, color[1]/255, color[2]/255)
        ret.append(c)
    return ret

cc = gen_c([c1,c2,c3,c4,c5])         

data = {'Mesh':[577], 
        'USTC impl':[186932-97692],
        'Ours':[20592],
        'Ours w/opt':[20388]}
df = pd.DataFrame(data)

font = {'family' : 'Times New Roman',
        'size'   : 12}


plt.rc('font', **font)

ax = sns.barplot(df,estimator="mean",errorbar=None, palette=cc)


plt.ylabel("Memory (KB)", fontsize=12)
plt.savefig('./exp/memory.png', dpi=300, bbox_inches='tight')
plt.show()